In [27]:
import requests

def get_response(url):
    cookies = {
        'fid': '18078',
        'lv': '1',
        'xxtenc': '412f54d525dd94167b18e70480127585',
        'createSiteSource': 'num3',
        'source': 'num3',
        'wfwEnc': 'F7F9C27BF373FB3E4D7A20D6E55807C8',
        '_uid': '199750697',
        'uf': 'b2d2c93beefa90dc9188507c473782d1fb6a2c25207d59ff2231b8cd478761ca2e15b8d2f6fb38f90fc9533a49dd6528f6a6e9008e9900b988b83130e7eb470494bd1582608cbc56aa8e8071455740ddd3d56c5b6a42238db6ab973de44b988b3fa46cb095462134aa2ebad65cd196bb',
        '_d': '1719822304820',
        'UID': '199750697',
        'vc': '4842034F9B9F91C0FA353C4B195A6F57',
        'vc2': '48BE4C984DF6538D0254D7ADE848CAE9',
        'vc3':  'LU7VBt%2FCGk1Y5qg7hD%2FEE1%2Ftds9v4G6TVKm2iS6m8ayhVkDFoy5wJrj5DGR7CzcBScrgy7AlgPFhJu3GBXr7GGsQv2PSGWfIamP%2B%2BBuvK7dLy2TM9hfZiSPNRvEycitSDTkXFa0lOEq18r%2FNwuCbARHlyw8zLGD2C5u0rI%2BkI04%3Dd96f869848bdc2221f2f08efd7cab984',
        'cx_p_token': 'f5f2850bd121270fbabb3c6a4d65df52',
        'p_auth_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1aWQiOiIxOTk3NTA2OTciLCJsb2dpblRpbWUiOjE3MTk4MjIzMDQ4MjIsImV4cCI6MTcyMDQyNzEwNH0.      0VtlwF_WUJSOD4U5FOb1rAYI9lS8GRUsYCZ-CI3Afv0',
        'DSSTASH_LOG': 'C_38-UN_5306-US_199750697-T_1719822304822',
        'browserLocale': 'en_US',
        'spaceFid': '18078',
        'spaceRoleId': '""',
        'k8s': '1719822328.927.88.267629',
        'jrose': 'B2D50F35C27D3EA8A288C24570A936D9.mooc-2061058409-5psw3',
        'route': '0a65fa708818ad1416475328b69707fd',
    }

    headers = {
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
       'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,en-GB;q=0.7,en-US;q=0.6',
       'Cache-Control': 'max-age=0',
       'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0',
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }



    response = requests.get(url, cookies=cookies, headers=headers)

    return response

In [28]:
import csv

# 假设 CSV 文件名为 url.csv，并且它位于当前工作目录下
csv_filename = 'url.csv'
url_list = []

with open(csv_filename, newline='', encoding='utf-8') as csvfile:
    # 创建 CSV 阅读器
    csvreader = csv.reader(csvfile)
    # 遍历 CSV 文件中的每一行
    for row in csvreader:
        # 将当前行的第一个元素（即 URL）添加到 url_list 中
        url_list.append(row[0])


responses = []

for url in url_list:
    response = get_response(url)  # 假设这是获取 URL 响应的函数
    responses.append(response)

In [29]:
import re 
from lxml import html

# 创建一个空列表来存储问题、选项和答案
questions_list = []

chapter_index = 0
# 遍历每个响应对象
for response in responses:
    # 编写正则表达式来匹配id="question数字"
    pattern = r'id="question\d+"'
    
    # 使用findall方法查找所有匹配的模式
    matches = re.findall(pattern, response.text)
    tree = html.fromstring(response.text)
    
    # 遍历每个匹配到的问题
    match_index = 1
    for match in matches:
        question_xpath = f'//*[@{match}]/h3'
        option_xpath = f'//*[@{match}]/ul/li'
        answer_xpath = f'//*[@{match}]/div/div[1]/span[1]'
        
        question = tree.xpath(question_xpath)[0].text_content()

        # 使用正则表达式匹配第一个括号内的内容
        match_question = re.search(r'\((.*?)\)', question)

        # 提取第一个括号内的内容
        question_type = match_question.group(1) if match else ""

        # 移除第一个括号内的内容并保留剩余的字符串
        question = re.sub(r'\((.*?)\)', '', question, 1)
        if (match_index<=9):
            question = question[3:]
        else:
            question = question[4:]

        question = re.sub(r'（', '(', question)
        question = re.sub(r'）', ')', question)
        options = [opt.text_content() for opt in tree.xpath(option_xpath)]

        answer = tree.xpath(answer_xpath)[0].text_content()
        answer = answer[11:]
        answer = answer.replace("对", "A")
        answer = answer.replace("错", "B")

        # 将问题、选项和答案存储为字典，并添加到列表中
        questions_list.append({
            'chapter': chapter_index,
            'question_type': question_type,
            'question': question,
            'options': options,
            'answer': answer
        })
        match_index += 1
    chapter_index += 1



In [30]:
# 将 questions_list 保存到 CSV 文件中
with open('完整题库.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['chapter', 'question_type', 'question', 'options', 'answer']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for question in questions_list:
        # 将列表转换为字符串存储
        question['options'] = ';'.join(question['options'])
        writer.writerow(question)